## Getting Started with Optimization.jl

In [4]:
# Import the package and define the problem to optimize
using Optimization, Zygote
rosenbrock(u, p) = (p[1] - u[1])^2 + p[2] * (u[2] - u[1]^2)^2
u0 = zeros(2)
p = [1.0, 100.0]

optf = OptimizationFunction(rosenbrock, AutoZygote())
prob = OptimizationProblem(optf, u0, p)

sol = solve(prob, Optimization.LBFGS())

retcode: Success
u: 2-element Vector{Float64}:
 0.9999997057368228
 0.999999398151528

### Import a different solver package

OptimizationOptimJL is a wrapper for [Optim.jl](#https://github.com/JuliaNLSolvers/Optim.jl) and OptimizationBBO is a wrapper for [BlackBoxOptim.jl](#https://github.com/robertfeldt/BlackBoxOptim.jl).

First let's use the NelderMead a derivative free solver from Optim.jl:

In [5]:
using OptimizationOptimJL
sol = solve(prob, Optim.NelderMead())

retcode: Success
u: 2-element Vector{Float64}:
 0.9999634355313174
 0.9999315506115275

BlackBoxOptim.jl offers derivative-free global optimization solvers that requrie the bounds to be set via `lb` and `ub` in the `OptimizationProblem`. Let's use the _BBOadaptivederand1binradiuslimited()_ solver:

In [8]:
using OptimizationBBO
prob = OptimizationProblem(rosenbrock, u0, p, lb = [-1.0, -1.0], ub = [1.0, 1.0])
sol = solve(prob, BBO_adaptive_de_rand_1_bin_radiuslimited())

retcode: MaxIters
u: 2-element Vector{Float64}:
 1.0
 0.9999999999999993

The solution from the original solver can always be obtained via `original`:

In [9]:
sol.original

BlackBoxOptim.OptimizationResults("adaptive_de_rand_1_bin_radiuslimited", "Max number of steps (10000) reached", 10001, 1.738093894447e9, 0.1510000228881836, BlackBoxOptim.ParamsDictChain[BlackBoxOptim.ParamsDictChain[Dict{Symbol, Any}(:RngSeed => 2798, :SearchRange => [(-1.0, 1.0), (-1.0, 1.0)], :TraceMode => :silent, :Method => :adaptive_de_rand_1_bin_radiuslimited, :MaxSteps => 10000),Dict{Symbol, Any}()],Dict{Symbol, Any}(:CallbackInterval => -1.0, :TargetFitness => nothing, :TraceMode => :compact, :FitnessScheme => BlackBoxOptim.ScalarFitnessScheme{true}(), :MinDeltaFitnessTolerance => 1.0e-50, :NumDimensions => :NotSpecified, :FitnessTolerance => 1.0e-8, :TraceInterval => 0.5, :MaxStepsWithoutProgress => 10000, :MaxSteps => 10000…)], 10103, BlackBoxOptim.ScalarFitnessScheme{true}(), BlackBoxOptim.TopListArchiveOutput{Float64, Vector{Float64}}(4.4373425918681914e-29, [1.0, 0.9999999999999993]), BlackBoxOptim.PopulationOptimizerOutput{BlackBoxOptim.FitPopulation{Float64}}(BlackBoxO

### Defining the objective function

Optimization.jl assumes that your objective function takes two arguments `objective(x, p)`

1. The optimization variables `x`.
2. Other parameters `p`, such as hyper parameters of the cost function. If you have no “other parameters”, you can safely disregard this argument. 

> Note: If your objective function is defined by someone else, you can create an anonymous function that just discards the extra parameters like this:

```julia
obj = (x, p) -> objective(x) # Pass this function into OptimizationFunction
```

### Controlling Gradient Calculations (Automatic Differentiation)

Notice that both of the above methods were derivative-free methods, and thus no gradients were required to do the optimization. However, often first order optimization (i.e., using gradients) is much more efficient. Defining gradients can be done in two ways. One way is to manually provide a gradient definition in the `OptimizationFunction` constructor. However, the more convenient way to obtain gradients is to provide an AD backend type.

For example, let's now use the OptimizationOptimJL `BFGS` method to solve the same problem. We will import the forward-mode automatic differentiation library (using ForwardDiff) and then specify in the `OptimizationFunction` to automatically construct the derivative functions using ForwardDiff.jl. This looks like: